In [ ]:
# preview and make tables from uploaded log files

In [38]:
try:
    influxdbconfig = dict(l.split()  for l in open("../influxdb_password.txt").readlines()  if l.strip())
except FileNotFoundError:
    print("Missing password file, creating blank one to fill in")
    fpass = open("../influxdb_password.txt", "w")
    fpass.write("influxdbuser      hanglog\n")
    fpass.write("influxdbpassword      password-goes-here\n")
    fpass.close()
    influxdbconfig["influxdbpassword"] = "password-goes-here"
if influxdbconfig["influxdbpassword"] == "password-goes-here":
    print("please set the password in the file influxdb_password.txt")
    #sys.exit()

import influxdb
client = influxdb.InfluxDBClient(host='influxdb.doesliverpool.xyz', port=8086, 
                                 username=influxdbconfig["influxdbuser"], password=influxdbconfig["influxdbpassword"])
print("databases present", client.get_list_database())
client.switch_database('hanglogdb')


databases present [{'name': 'ming_default'}, {'name': '_internal'}, {'name': 'julianhousedb'}, {'name': 'rosebankdb'}, {'name': 'hanglogdb'}]


In [39]:
import pandas

k = client.query("SELECT * FROM hanglog_index")
pI = pandas.DataFrame(k.get_points())
pI = pI.drop(columns=["logchannel", "time"], errors="ignore")
pI

,devicename,fdir,fname,ft0,ft1,rectypespresent,uploaddate
0,jgt1,dd_2020_09_20_160906,hdata-2020-09-20_16-09-06.log,2020-09-20T16:09:06.855000,2020-09-20T16:12:47.697000,aZ aA aF aQ aV,2021-04-03
1,jgt1,dd_2020_10_11_111652,hdata-2020-10-11_11-16-52.log,2020-10-11T11:16:52.498000,2020-10-11T11:20:31,aZ aA aF aQ aV,2021-04-03
2,jgt1,dd_2020_12_11_101346,hdata-2020-12-11_10-13-46.log,2020-12-11T10:13:46.256237,2020-12-11T10:14:28.459237,Y Z U aZ aA aF,2021-04-03
3,jgt1,dd_2021_04_02_172718,hdata-2021-04-02_17-27-18.log,2021-04-02T17:27:18.106000,2021-04-02T17:47:10.500000,aZ aA aF aQ aV,2021-04-03
4,sopernew,dd_2021_04_03_194748,hdata-2021-04-03_19-47-48.log,2021-04-03T19:47:47.976125,2021-04-03T19:48:22.051125,W U M aF aA aZ,2021-04-03
5,sopernewmod,dd_2021_04_03_194748,hdata-2021-04-03_19-47-48.log,2021-04-03T19:47:48.029125,2021-04-03T19:48:22.249125,W U M aA aZ aF,2021-04-03


In [47]:
row = pI.iloc[5]

In [48]:
row.rectypespresent.split()

['W', 'U', 'M', 'aA', 'aZ', 'aF']

In [49]:
row.fdir

'dd_2021_04_03_194748'

In [50]:
fd = { }
for kk in row.rectypespresent.split():
    print("fetching", kk)
    k = client.query("SELECT * FROM %s where fdir='%s'" % (kk, row.fdir))
    pK = pandas.DataFrame(k.get_points())
    if len(pK) != 0:
        ds = pandas.DatetimeIndex(pK.time)
        pK = pK.set_index(ds)
        pK.index.name = ""
        pK = pK.drop(columns=["time", "logchannel", "fdir", "device", "fname"], errors="ignore")
        fd[kk] = pK
        print("   ", len(pK), "records")

fetching W
fetching U
fetching M
fetching aA
fetching aZ
    170 records
fetching aF
    170 records


In [24]:
fd["aQ"]

,alt,lat,lng,u
,,,,
2020-10-11 11:17:14+00:00,124.9,53.419432,-2.969562,40634000
2020-10-11 11:17:52+00:00,125.0,53.419432,-2.969565,40672000
2020-10-11 11:18:41+00:00,125.1,53.419443,-2.969570,40721000
2020-10-11 11:18:43+00:00,125.1,53.419463,-2.969567,40723000
2020-10-11 11:18:44+00:00,125.2,53.419475,-2.969568,40724000
2020-10-11 11:18:45+00:00,125.1,53.419488,-2.969577,40725000
2020-10-11 11:18:46+00:00,125.1,53.419502,-2.969582,40726000
2020-10-11 11:18:56+00:00,125.3,53.419493,-2.969580,40736000
2020-10-11 11:18:57+00:00,125.3,53.419483,-2.969582,40737000


In [37]:
(10001*100),0xFFFFFF

(1000100, 16777215)